In [ ]:
!pip install ultralytics
!pip install roboflow
!pip install -U ipywidgets

In [ ]:
import os
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
# yolo predict model=yolo11n.pt source='https://ultralytics.com/images/bus.jpg'

In [ ]:
#Remove all files

import os
import shutil

# Specify the folder path
folder_path = '/kaggle/working/'

# Remove all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove file or link
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove directory and its contents
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

print("All files have been removed from the folder.")


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11x-seg.pt")

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="xTo8OzKt3gqUQENvd7gp")
project = rf.workspace("damage-sverity").project("flowers-krbsa")
version = project.version(2)
dataset = version.download("yolov11")

# Training on GPU (Option 01)

In [ ]:
# Train the model
train_results = model.train(
    data="/kaggle/working/Flowers-2/data.yaml",  # path to dataset YAML
    epochs=600,  # number of training epochs
    imgsz=640,  # training image size
    device=0,
    batch=4,
    workers=4,
    seed=101,
    
    # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

# Training on GPU (Option 02)

In [ ]:
!yolo segment train model=yolo11x-seg.pt data='/data/data_modified.yaml' workers=8 batch=8 epochs=300 seed=101 device=0 save_period=40 #patience=600 #name=yolov8_car_damage_296_Classes

# Continue Training

In [ ]:
!yolo segment train model=weights/best.pt data='/data/data_modified.yaml' resume=True workers=8 batch=8 epochs=600 seed=101 device=0 save_period=40 #patience=600 #name=yolov8_car_damage_296_Classes

# Saving the model

In [ ]:
# Export the model to ONNX format
path = model.export(format="onnx")  # return path to exported model

# Inference (Option 01)

In [ ]:
# Evaluate model performance on the validation set
metrics = model.val()

# Perform object detection on an image
results = model("/kaggle/working/Flowers-2/valid/images/IMG_3166_png_jpg.rf.192d29b990828bf0a453106caea9490a.jpg")
results[0].show()

# Inference (Option 02)

In [ ]:
!yolo predict model=yolo11x-seg.pt source='/images/IMG_3190a.jpg'